# DECRETO CER

### AUTENTIFICAZIONE E CREAZIONE CHAT

In [1]:
import requests


BASE_URL = "http://localhost:3000"
url = f"{BASE_URL}/api/v1/auths/signin"
payload = {
    "email": "eleonora.fabris@dbagroup.it", 
    "password": "Scoiattolo1!"
}
response = requests.post(url, json=payload)
token = response.json()["token"]

url = f"{BASE_URL}/api/v1/models"
JWT_TOKEN = token

HEADERS_JSON = {
    "Authorization": f"Bearer {JWT_TOKEN}",
    "Content-Type": "application/json"
}
HEADERS_FILE = {
    "Authorization": f"Bearer {JWT_TOKEN}"
}

params = {
        "max_new_tokens": 50, 
        "temperature": 0.7, # se 0.0 risponde sempre allo stesso modo, estremamente preciso
        "top_p": 0.9, # indica la variabilità del vocabolario
    }

response = requests.get(url, headers=HEADERS_JSON)

if response.status_code == 200:
    print("Autenticazione riuscita!")
    print(response.json()) # lista modelli disponibili
else:
    raise Exception(f"Errore nell'autenticazione: {response.status_code}")

Autenticazione riuscita!
{'data': [{'id': 'gemma3:4b', 'name': 'gemma3:4b', 'object': 'model', 'created': 1768290651, 'owned_by': 'ollama', 'ollama': {'name': 'gemma3:4b', 'model': 'gemma3:4b', 'modified_at': '2025-12-22T16:59:15.153912+01:00', 'size': 3338801804, 'digest': 'a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3', 'families': ['gemma3'], 'parameter_size': '4.3B', 'quantization_level': 'Q4_K_M'}, 'connection_type': 'local', 'urls': [0]}, 'connection_type': 'local', 'tags': [], 'actions': [], 'filters': []}, {'id': 'falcon3:7b', 'name': 'falcon3:7b', 'object': 'model', 'created': 1768290651, 'owned_by': 'ollama', 'ollama': {'name': 'falcon3:7b', 'model': 'falcon3:7b', 'modified_at': '2025-12-22T16:51:27.8742221+01:00', 'size': 4569738977, 'digest': '472ea1c89f64f932368963bfb8c7f54bd797f2161932cf0a72766a587b2e586d', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families

In [2]:
import requests
import json


pdf_file_path = r"C:\DEV\LAB\electrical-energy-billing\Decreto CER.pdf"
with open(pdf_file_path, "rb") as f:
    files = {"file": ("Decreto.pdf", f, "application/pdf")}
    response = requests.post(f"{BASE_URL}/api/v1/files/?process=true", files=files, headers=HEADERS_FILE)

if response.status_code not in [200, 201]:
    raise Exception(f"Errore nel caricamento del file: {response.status_code} - {response.text}")

file_data = response.json()
file_id = file_data.get("id")

with requests.get(
    f"{BASE_URL}/api/v1/files/{file_id}/process/status?stream=true",
    headers=HEADERS_FILE,
    stream=True
) as r:
    if r.status_code != 200:
        raise Exception(f"Errore status SSE (Server-Sent Events): {r.status_code} - {r.text}")
    for line in r.iter_lines(decode_unicode=True):
        if not line:
            continue
        if line.startswith("data:"):
            data = json.loads(line.replace("data:", "").strip())
            status = data.get("status")
            if status == "completed":
                print(f"Caricamento del file {file_id} completato.")
                break
            if status in ("failed", "error"):
                raise Exception("Processing fallito")

Caricamento del file 349bc688-aff5-425b-8550-ee30b6039d69 completato.


In [3]:
from pypdf import PdfReader  

reader = PdfReader(pdf_file_path)
pdf_text = "\n".join([page.extract_text() or "" for page in reader.pages])


In [4]:
import requests
import uuid


initial_assistant_id = str(uuid.uuid4())

payload_new_chat = {
    "chat": {
        "title": "Lettura Decreto CER",
        "models": ["falcon3:7b"], 
        "params": params
    }
}
url_new_chat = f"{BASE_URL}/api/v1/chats/new"
response = requests.post(url_new_chat, json=payload_new_chat, headers=HEADERS_JSON)
if response.status_code not in [200, 201]:
    raise Exception(f"Errore nella creazione chat: {response.status_code} - {response.text}")

chat_data = response.json()
chat_id = chat_data["id"]
current_id = initial_assistant_id
print("Nuova chat creata:", chat_id)

url_get_chat = f"{BASE_URL}/api/v1/chats/{chat_id}"
chat_info = requests.get(url_get_chat, headers=HEADERS_JSON).json()

Nuova chat creata: c73a3640-d967-4881-b6fd-4ec13c5a2bd3


### INVIO MESSAGGIO

In [ ]:
messages = [
    {"role": "user", 
     "content": f"""
        Analizza ESCLUSIVAMENTE il contenuto del PDF: {pdf_text}. 
        Riassumi in modo chiaro e coinciso cosa contiene il documento.
        """
    }
]


In [6]:
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages,
    "files": [
        {
            "id": file_id,
            "type": "file"
        }
    ]
}
url_completion = f"{BASE_URL}/api/chat/completions"
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]

print(f"Risposta: {answer}")

Risposta: ### Riassunto del Documento

Il documento è una richiesta di accesso a documenti legali che riguardano l'accesso ai meccanismi incentivanti per impianti di biomassa e erbacee. Questi meccanismi sono integrati con quanto disposto dalla legge n. 211 del 28 dicembre 2015, caratterizzata come "Collegato Ambientale". Il documento è composto da un'introduzione che richiede accesso a documenti legali, seguiti di due tabelline:

- **Tabella 1 (Non fornita nel tuo input)**: Non fornita.
- **Tabella 2**: Elenco dei sottoprodotti e prodotti utilizzabili negli impianti a biomassa, dividuto in parti A (sottoprodotti provenienti da diversi attività come agricola, forestale, alimentare) e parte B (prodotti specifici come specie erbacee annuali e poliennali, e arvore).

### Domande Risposte Utilizzando Informazioni dal Documento

#### Domanda 1: Quali sono i sottoprodotti utilizzabili negli impianti a biomassa provenienti da attività agricola?

**Risposta**: I sottoprodotti utilizzabili negl

In [7]:
messages.append({
    "role": "user",
    "content": f"""Secondo il SOLO contenuto del PDF: Decreto CER (Comunità Energetiche Rinnovabili), quali sono i soggetti beneficiari, 
            le tipologie di imprese e gli enti pubblici o privati (nel senso giuridico) che possono partecipare alle configurazioni di energia condivisa? """
})
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages
}
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"Risposta: {answer}")


Risposta: Secondo il SOLO contenuto del PDF relativo al Decreto CER (Comunità Energetiche Rinnovabili), le condizioni per partecipare alle configurazioni di energia condivisa sono state stabilite con l'obiettivo di promuovere la riutilizzazione delle risorse e l'uso di energia rinnovabile.

I soggetti beneficiari sono principalmente:

1. **Enti Pubblici**: Enti locali, regioni, città metropolitane, autorità locali e qualsiasi altro ente pubblico che possa essere responsabile per l'approvvigionamento energetico o la gestione di infrastrutture energetiche.

2. **Enti Privati**: Impresi e organizzazioni private che siano interessati a partecipare alle comunità energiche rinnovabili, con il compito di sviluppare e gestire sistemi di energia condivisa.

3. **Organizzazioni Non Profit**: Enti non profit che operano in ambito energetico, specialmente quelli che lavorano per la sostenibilità ambientale e l'economia locale.

4. **Consorti e Società Collettive**: Gruppi di enti o imprese che si 

In [ ]:
messages.append({
    "role": "user",
    "content": f""" Secondo il SOLO contenuto del PDF, rispondi solamente a questa domanda:
        definisci impianto alimentato da fonti rinnovabili """
})
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages
}
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"Risposta: {answer}")


Risposta: Un impianto alimentato da fonti rinnovabili è un sistema tecnologico che utilizza energie provenienti da fonti rinnovabili come l'acqua, il vento, il sole, la biomassa, il calore terrestre e il mare per produrre energia utilizzabile. Queste fonti sono considerate rinnovabili perché possono essere ricarriciate in modo naturale senza rischi per il clima o per la biodiversità. 

Gli impianti alimentati da fonti rinnovabili includono, ma non si limitano a, fotovoltaici (capovoltaici), aerogeneratori, turbina idrauliche, cicli di calore solari, eletrotermici, e impianti eolici. Questi sistemi convertiscono la energia fisica o chimica delle fonti rinnovabili in forme di energia utilizzabile come luce, calore, elettricità, che possono essere utilizzati per alimentare edifici, trasporti, industria e servizi varii.

In quanto alla domanda riguardante il Decreto CER (Comunità Energetiche Rinnovabili), i soggetti beneficiari sono le imprese private, gli enti pubblici, ed organizzazioni 

In [ ]:
messages.append({
    "role": "user",
    "content": f"""Secondo il SOLO contenuto del PDF, , rispondi solamente a questa domanda:
        quali sono i compiti e gli incarichi del GSE?"""
})
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages
}
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"Risposta: {answer}")


In [ ]:
messages.append({
    "role": "user",
    "content": f"""Secondo il SOLO contenuto del PDF, rispondi solamente a questa domanda: 
        quale requisito demografico devono soddisfare i comuni per permettere alle comunità energetiche e 
        alle configurazioni di autoconsumo collettivo di accedere ai contributi in conto capitale del PNRR? """
})
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages
}
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"Risposta: {answer}") # popolazione inferiore 5000 abitanti


Risposta: Impianto alimentato da fonti rinnovabili: Un impianto alimentato da fonti rinnovabili è un sistema energetico che utilizza energie provenienti da fonti naturali e non depletibili, come l'energia solare, l'eolica, l'acqua e la biomassa. Questi impianti contribuiscono a ridurre la dipendenza dai combustibili tradizionali e a diminuire gli impatti ambientali associati alla produzione di energia.

Quanto al requisito demografico per i comuni che vogliono accedere ai contribui in conto capitale del PNRR per le comunità energetiche e le configurazioni di autoconsumo collettivo, secondo il SOLO contenuto del PDF, i comuni devono avere una popolazione minima di 5.000 abitanti per essere considerati per l'accesso a questi finanziamenti.


In [ ]:
messages.append({
    "role": "user",
    "content": f"""Secondo il SOLO contenuto del PDF, rispondi solamente a questa domanda: 
        qual è la data limite entro cui gli impianti che beneficiano del contributo PNRR devono entrare in esercizio, secondo l'articolo 7? """
})
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages
}
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"Risposta: {answer}") # 31 dic 2027

Risposta: Secondo il SOLO contenuto del PDF, rispondi solamente a questa domanda: 

Qual è la data limite entro cui gli impianti che benefici di un contributo PNRR devono entrare in esercizio, secondo l'articolo 7? La data limite è il 31 dicembre 2027.


In [20]:
messages.append({
    "role": "user",
    "content": f"""Secondo il SOLO contenuto del PDF, rispondi solamente a questa domanda: 
        secondo l'articolo 3, qual è la potenza nominale massima che un singolo impianto
        può avere per accedere agli incentivi? """
})
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.4,
        "top_p": 0.85
    },
    "messages": messages
}
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"Risposta: {answer}") # non superiore a 1 MW

Risposta: Secondo il SOLO contenuto del PDF, rispondi solamente a questa domanda: 
        Secondo l'articolo 3, la potenza nominale massima che un singolo impianto può avere per accedere agli incentivi sotto forma di tariffa premio è 10 MW.


### CANCELLA CHAT E PDF

In [ ]:
import subprocess
import requests


url_completed = f"{BASE_URL}/api/chat/completed"
close_chat = requests.post(url_completed, headers=HEADERS_JSON)


file_path = file_data["path"]

response = requests.delete(f"{BASE_URL}/api/v1/chats/{chat_id}", headers=HEADERS_FILE)

if response.status_code not in (200, 204):
    raise Exception(f"Errore cancellazione chat: {response.status_code} - {response.text}")
print(f"Chat {chat_id} eliminata correttamente.")

result = subprocess.run(["docker", "exec", "open-webui", "rm", "-f", file_path], 
    capture_output=True, text=True)

if result.returncode == 0:
    print(f"File PDF {file_id} eliminato correttamente.")
else:
    print("Errore eliminazione file:", result.stderr)
